# <span style="color: black; font-family: Babas Neue; font-size: 3em;">Tutoriel 1.0</span>
<hr height="3" style="color:purple;background-color:purple;"></hr>
<span style="color: black; font-family: Babas Neue; font-size: 1em;">Extraction d'articles depuis les archives de La Presse</span>

<br>
<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Bienvenue à notre premier tutoriel!** Nous travaillerons ici sur un script qui nous permettra de parcourir les archives du quotidien La Presse. Nous avons choisi La Presse pour notre tutoriel car ses archives respectent une forme standard, ce qui permet au code d'être facilement repensé pour un autre projet d'extraction de données. <br><br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Pour mener à bien ce projet, nous utiliserons le langage python à l'intérieur de l'environnement Anaconda. [Anaconda](https://www.continuum.io/downloads) est un écosystème d'applications, de paquets et de modules pour les sciences des données. Il est fort probable que ce que vous recherchez soit contenu dans l'environnement Anaconda de base, mais, au besoin, il est toujours possible d'installer manuellement un module ou une application.
<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Si vous souhaitez exécuter les codes par vous-même et explorer, veuillez installer la suite Anaconda pour python 3.6. Sinon, vous pouvez aussi lire ce fichier - sans l'exécuter - comme une sorte de rapport, car il est possible d'y intégrer des notes ou des scripts ainsi que d'afficher les résultats.
</span></div>

> <div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Pour notre projet, nous utiliserons les modules suivant (venant par défaut avec l'environnement anaconda)<br><br>
- [urllib.request](https://docs.python.org/3/library/urllib.request.html) (Pour envoyer les requêtes URL)<br>
- [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) (Abbrévié 'bs4'. Sert à traiter le contenu URL)<br>
- [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html) (Analyse de bases de données en langage python. L'équivalent d'Excel en ligne de commande. Utilisé pour toute les manipulations de bases de données)<br>
- [numpy](http://www.numpy.org/) (Programmation scientifique. Pandas est construit par-dessus l'infrastructure de NumPy, mais ce dernier contient beaucoup de fonctions et algorithmes qui lui sont propres.)
</span></div>

> <div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
De plus, nous utiliserons le module [newspaper](http://newspaper.readthedocs.io/en/latest/), développé en open-source par Lucas Ou-Yang. Celui-ci ne fait pas parti de l'écosystème Anaconda, nous devrons donc l'installer manuellement. <br><br>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Voici les grandes étapes de notre extraction :<br><br>1) Générer une liste de dates. <br>2) Générer, par une boucle, une liste de liens vers les archives.<br> 3) Générer, en interrogeant les archives, une liste de liens vers les articles. <br> 4) Interroger les liens vers les articles afin d'en extraire le contenu. <br><br><br>***Afficher la version de la suite anaconda***</span></div>

###### <span style="color: black; font-family: Babas Neue; font-size: 2em;">0.0 Exemple</span>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Ce document contient des zones de texte (comme ici), des zones de script (vous pouvez les voir avec une invite de commande 'In [x]') situé sur la gauche. La réponse (*output*) du script figure en bas des zones de script.<br><br><br>***Afficher la version de la suite anaconda***</span></div>

In [1]:
!conda --version

conda 4.3.23


<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>Dans la zone de script, les mots suivis par un # sont des commentaires pour vous aider à comprendre. Ils n'ont aucun effet sur votre code<br><br>*** Afficher le résultat d'une équation***</span></div>

In [2]:
x = 10                        # Créer une variable 'x' avec le nombre entier 10 comme valeur
equation_ex = x*(4+1/(2*3)-2) # Les opérateurs mathématiques sont semblables (* = multi, / = div)
print (equation_ex)           # Afficher la variable 'equation_ex', dont la valeur est le résultat de l'équation

21.66666666666667


###### <span style="color: black; font-family: Babas Neue; font-size: 2em;">1. Importation</span>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Pour commencer, nous allons importer les modules que nous avons déjà dans l'environnement Anaconda, puis installer le module newspaper.<br><br><br>***1.1 Importation des modules***</span></div>

In [3]:
import urllib.request
import bs4
import numpy as np  # 'as' signifie que que nous allons abbrévier le nom, puisque ce sont des modules souvent utilisés
import pandas as pd # Avant : pandas.DataFrame(...), Maintenant : pd.DataFrame(...)

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 1.2.1 Installer newspaper (exécuter dans un shell ou terminal) ***</span></div>


In [4]:
!pip install newspaper3k

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Comme j'ai déjà installé le module manuellement, la machine me répond que les exigences sont déjà satisfaites.

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 1.2.2 Importer newspaper***</span></div>


In [26]:
import newspaper

###### <span style="color: black; font-family: Babas Neue; font-size: 2em;">2. Création d'une liste de date</span>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Nous allons  générer une liste des dates qui couvrent notre étude. Nous le faisons pour plusieurs raisons, notamment parce que les addresses URL de La Presse intègrent les dates et parce que, de toute façon, nous allons entrer la donnée 'DATE' dans la base de données que nous allons créer.<br><br><br>***2.1 Génération d'une liste de date avec numpy (np) et pandas (pd)***</span></div>

In [6]:
array_of_time = pd.to_datetime(np.arange('2012-12-01', '2013-07-01', dtype='datetime64[D]'))
print (array_of_time)

DatetimeIndex(['2012-12-01', '2012-12-02', '2012-12-03', '2012-12-04',
               '2012-12-05', '2012-12-06', '2012-12-07', '2012-12-08',
               '2012-12-09', '2012-12-10',
               ...
               '2013-06-21', '2013-06-22', '2013-06-23', '2013-06-24',
               '2013-06-25', '2013-06-26', '2013-06-27', '2013-06-28',
               '2013-06-29', '2013-06-30'],
              dtype='datetime64[ns]', length=212, freq=None)


<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Les '...' que vous voyez dans la réponse signifient que vous ne voyez pas l'entièreté de la liste. On peut constater, à cette fin, que la liste contient 212 entrées, ce qui est exact si l'on souhaite intégrer les jours du 1er décembre 2012 à la dernière journée de juin 2013 inclusivement. <br><br> Non seulement nous avons dressé rapidement une liste, mais nous avons également formatté les données pour qu'elles soient perçues comme un objet de type 'date' plutôt que comme une chaîne de caractères normale, (Comme 'Roger'). Ces données contiennent donc maintenant des attributs propres aux dates.

In [7]:
print (array_of_time[0]) # Imprimer la première donnée de notre liste de date (liste[index])
print ('2012-12-01 00:00:00') # Imprimer une chaîne de caractères standard qui semble identique

2012-12-01 00:00:00
2012-12-01 00:00:00


In [8]:
print (array_of_time[0].year) # Imprimer l'année
print (array_of_time[0].day) # Imprimer le jour
print (array_of_time[0].month) # Imprimer le mois
print (array_of_time[0].weekday_name)  # Imprimer le nom de jour dans la semaine
print ('.. Il existe beaucoup plus de fonctions! ..')

2012
1
12
Saturday
.. Il existe beaucoup plus de fonctions! ..


In [9]:
print ('2012-12-01 00:00:00'.year) # Essayer de faire une même fonction avec la chaîne de caractère

AttributeError: 'str' object has no attribute 'year'

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Nous pouvons constater ici qu'une chaîne de caractères standard ne possède pas l'attribut 'année', puisqu'elle donne une réponse d'erreur. Notre première donnée de la liste est convertie comme un objet de type 'date', ce qui la permet de posséder cet attribut. Vous voyez donc l'intérêt de *formatter* les données en plus de les générer à grande échelle. 

###### <span style="color: black; font-family: Babas Neue; font-size: 2em;">3. Générer une liste de liens URL vers les archives</span>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Le système d'archives de La Presse fonctionne ainsi :
<br><br>
http://www.lapresse.ca/archives / années / mois / jour .php
<br><br>
À partir de notre première liste de date, nous allons en générer une liste de liens, qui va contenir les adresses URL des archives. Il ne s'agit pas de notre cible final, c'est-à-dire qu'il faudra ensuite aller à l'intérieur de ses liens pour produire une troisième liste, qui est constituée des adresses URL des articles.

<br><br><br>***3.1 Définition de la souche***</span></div>

In [10]:
root_url = 'http://www.lapresse.ca/archives'
url_end = '.php'

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 3.2 Créer une liste vide***</span></div>

In [11]:
archives_url = []

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 3.3 Peupler la liste à partir d'une boucle ***</span></div>

In [12]:
for x in range(len(array_of_time)):   # Pour tout les éléments de la première liste, 
    day = array_of_time[x].day        # capturer les données day, month, year
    month = array_of_time[x].month    
    year = array_of_time[x].year
    archives_url.append('{}/{}/{}/{}{}'.format(root_url,year,month,day,url_end)) # Stocker l'URL dans la nouvelle liste

In [13]:
print(archives_url)

['http://www.lapresse.ca/archives/2012/12/1.php', 'http://www.lapresse.ca/archives/2012/12/2.php', 'http://www.lapresse.ca/archives/2012/12/3.php', 'http://www.lapresse.ca/archives/2012/12/4.php', 'http://www.lapresse.ca/archives/2012/12/5.php', 'http://www.lapresse.ca/archives/2012/12/6.php', 'http://www.lapresse.ca/archives/2012/12/7.php', 'http://www.lapresse.ca/archives/2012/12/8.php', 'http://www.lapresse.ca/archives/2012/12/9.php', 'http://www.lapresse.ca/archives/2012/12/10.php', 'http://www.lapresse.ca/archives/2012/12/11.php', 'http://www.lapresse.ca/archives/2012/12/12.php', 'http://www.lapresse.ca/archives/2012/12/13.php', 'http://www.lapresse.ca/archives/2012/12/14.php', 'http://www.lapresse.ca/archives/2012/12/15.php', 'http://www.lapresse.ca/archives/2012/12/16.php', 'http://www.lapresse.ca/archives/2012/12/17.php', 'http://www.lapresse.ca/archives/2012/12/18.php', 'http://www.lapresse.ca/archives/2012/12/19.php', 'http://www.lapresse.ca/archives/2012/12/20.php', 'http://

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Générer une telle liste nécessite d'examiner la structure du site web en question, de comprendre sa logique, de retenir les constantes (le début du site et le type '.php, .html') et générer les extensions souhaitées par la suite. Dans ce cas, il s'agit de l'année, du mois et du jour.

###### <span style="color: black; font-family: Babas Neue; font-size: 2em;">4. Explorer les liens des archives</span>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Nous devons interroger la liste de liens des archives dans le but d'extraire les liens vers les articles. Chaque lien des archives est donc à la fois la cible d'une requête et le conteneur de d'autres liens, qui feront l'objet d'une requête antérieur (pour accéder au contenu de chaque article). <br><br><br>***4.1 Interroger un site (nous allons prendre le lien du 11 janvier 2013)***</span></div>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 4.1.1 Envoyer une requête au serveur avec urllib.request***</span></div>

In [17]:
print (archives_url[41]) # Vérifier que l'index de la liste correspondent à ce que nous voulons

http://www.lapresse.ca/archives/2013/1/11.php


In [18]:
request = urllib.request.Request(url = archives_url[41], data=None, headers=
                             {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) \
                             AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 4.1.2 Ouvrir un objet BeautifulSoup4 avec la réponse de la requête.*** <br><br>Dit autrement, nous passons de la demande d'information à réponse obtenue par le serveur. bs4 va nous afficher le contenu de la réponse</span></div>

In [19]:
source_code = bs4.BeautifulSoup(urllib.request.urlopen(request).read(),'lxml')

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">***<br><br>4.1.2 Utiliser la méthode prettify pour afficher le contenu. Il s'agit du code source de la page.***</span></div>

In [20]:
print(source_code.prettify)

<bound method Tag.prettify of <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<!-- Tout le site sauf l accueil et Rmon1 --><html xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<!-- / Tout le site sauf l accueil et Rmon1 -->
<!-- Tout le site -->
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>LaPresse.ca | Actualités, Arts, International, Débats, Sports, Vivre, Voyage</title>
<meta content="LaPresse.ca | Actualités, Arts, International, Débats, Sports, Vivre, Voyage" name="description"/>
<meta content="Actualités, Arts, International, Débats, Sports, Vivre, Voyage" name="keywords"/>
<meta content="LaPresse.ca | Actualités, Arts, International, Débats, Sports, Vivre, Voyage" property="og:title"/>
<meta content="LaPresse.ca" property="og:description"/>
<meta content="LaPresse.ca | Act

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> L'intérêt est d'extraire un seul site pour commencer et de voir son contenu afin essayer d'en déduire la structure interne commune. Si vous observez bien, vous verrez que la liste des articles est balisée avec un code 'ul class="square square-spread"', qui signifie *unordered list*. Cela est très significatif, car nous avons exactement droit ici à une telle liste. Probablement que cette balise est donc utilisée comme standard par La Presse. Nous tenterons de l'utiliser pour tout les autres liens.  

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">***<br>4.1.3 Convertir le contenu en chaîne de caractères et tronquer la sélection souhaitée.***</span></div>

In [21]:
str_from_code = str(source_code)
start_idx = str_from_code.find('<ul class="square square-spread">')
skimed_str = str_from_code[start_idx:str_from_code.find('</ul>',start_idx)]
print (skimed_str)

<ul class="square square-spread">
<li>(05:06) <a alt="C'est arrivé un 11 janvier" href="http://www.lapresse.ca/le-droit/actualites/ephemerides/201301/04/01-4608380-cest-arrive-un-11-janvier.php" title="C'est arrivé un 11 janvier">C'est arrivé un 11 janvier</a></li>
<li>(05:00) <a alt="La LNH s'éloigne de Québec" href="http://www.lapresse.ca/le-soleil/opinions/carrefour/201301/08/01-4609376-la-lnh-seloigne-de-quebec.php" title="La LNH s'éloigne de Québec">La LNH s'éloigne de Québec</a></li>
<li>(07:02) <a alt="Seul devant la cour, un bon choix?" href="http://www.lapresse.ca/la-tribune/archives/la-nouvelle/actualites/201301/09/01-4609791-seul-devant-la-cour-un-bon-choix.php" title="Seul devant la cour, un bon choix?">Seul devant la cour, un bon choix?</a></li>
<li>(11:06) <a alt="Impair, un collectif inspiré par l'art de rue" href="http://www.lapresse.ca/la-tribune/archives/la-nouvelle/scene-culturelle/201301/09/01-4609793-impair-un-collectif-inspire-par-lart-de-rue.php" title="Impair, u

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Nous pouvons observer que les liens qui nous intéresse sont précédés de la code 'href' et termine toujours par leurs extensions '.php'. À partir de là, il ne reste que quelques manipulations pour accéder générer une liste des liens.

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">***<br>4.1.4 Diviser le texte par le mot-clé 'href' et lister les éléments. Enlever le premier index car il ne s'agit pas d'un site.***</span></div>

In [22]:
php_list = skimed_str.split('href="')[1:]
php_list

['http://www.lapresse.ca/le-droit/actualites/ephemerides/201301/04/01-4608380-cest-arrive-un-11-janvier.php" title="C\'est arrivé un 11 janvier">C\'est arrivé un 11 janvier</a></li>\n<li>(05:00) <a alt="La LNH s\'éloigne de Québec" ',
 'http://www.lapresse.ca/le-soleil/opinions/carrefour/201301/08/01-4609376-la-lnh-seloigne-de-quebec.php" title="La LNH s\'éloigne de Québec">La LNH s\'éloigne de Québec</a></li>\n<li>(07:02) <a alt="Seul devant la cour, un bon choix?" ',
 'http://www.lapresse.ca/la-tribune/archives/la-nouvelle/actualites/201301/09/01-4609791-seul-devant-la-cour-un-bon-choix.php" title="Seul devant la cour, un bon choix?">Seul devant la cour, un bon choix?</a></li>\n<li>(11:06) <a alt="Impair, un collectif inspiré par l\'art de rue" ',
 'http://www.lapresse.ca/la-tribune/archives/la-nouvelle/scene-culturelle/201301/09/01-4609793-impair-un-collectif-inspire-par-lart-de-rue.php" title="Impair, un collectif inspiré par l\'art de rue">Impair, un collectif inspiré par l\'art d

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">***<br>4.1.5 Modifier les éléments de la liste pour qu'ils s'arrêtent à l'extension '.php' ***</span></div>

In [23]:
for x in range(len(php_list)):
    php_list[x] = php_list[x][:php_list[x].find('.php')+4]
php_list

['http://www.lapresse.ca/le-droit/actualites/ephemerides/201301/04/01-4608380-cest-arrive-un-11-janvier.php',
 'http://www.lapresse.ca/le-soleil/opinions/carrefour/201301/08/01-4609376-la-lnh-seloigne-de-quebec.php',
 'http://www.lapresse.ca/la-tribune/archives/la-nouvelle/actualites/201301/09/01-4609791-seul-devant-la-cour-un-bon-choix.php',
 'http://www.lapresse.ca/la-tribune/archives/la-nouvelle/scene-culturelle/201301/09/01-4609793-impair-un-collectif-inspire-par-lart-de-rue.php',
 'http://www.lapresse.ca/la-tribune/archives/la-nouvelle/actualites/201301/09/01-4609795-la-consigne-sociale-en-plein-emergence.php',
 'http://www.lapresse.ca/le-droit/actualites/chroniques/denis-gratton/201301/10/01-4609974-evangeline-gabriel-et-le-chat.php',
 'http://www.lapresse.ca/le-soleil/opinions/carrefour/201301/10/01-4610077-nadeau-dubois-futur-lieutenant-gouverneur.php',
 'http://www.lapresse.ca/debats/editoriaux/francois-cardinal/201301/10/01-4610192-les-racines-de-la-colere.php',
 'http://www.

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"><br>*** 4.2 Application de l'extraction à tout les liens.***</span></div>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Pour obtenir tout les articles écris par la La Presse du 1er décembre au 30 juin, il suffit de généraliser ce que l'on vient de faire aux autres liens des archives et stocker les nouveaux dans une nouvelle variable. Vous pouvez exécuter le code ci-dessous, mais je vais le désactiver par défaut, car, pour le bien du tutoriel, je ne traiterai que d'une seule journée, soit celle que je viens de faire.  

In [24]:
#metalist = []
#for x in range(len(archives_url)):
#
#    request = urllib.request.Request(url = archives_url[x], data=None, headers=
#                             {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) \
#                             AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
#
#    source_code = bs4.BeautifulSoup(urllib.request.urlopen(request).read(),'lxml')
#    str_from_code = str(source_code)
#    start_idx = str_from_code.find('<ul class="square square-spread">')
#    skimed_str = str_from_code[start_idx:str_from_code.find('</ul>',start_idx)]
#    php_list = skimed_str.split('href="')[1:]
#    
#    for y in range(len(php_list)):
#        php_list[y] = php_list[y][:php_list[y].find('.php')+4]
#
#    metalist.extend(php_list)
    
#metalist

###### <span style="color: black; font-family: Babas Neue; font-size: 2em;">5. Explorer les liens des articles</span>

<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;"> Notre liste de liens vers les articles ayant été générée, nous allons pouvoir maintenant extraire d'elles l'information qui nous intéresse. Souvent, notre but est de rechercher quelque chose de particulier, comme dans notre situation où nous recherchons les articles ayant la mention IDLE NO MORE dans le texte. Il ne s'agit pas d'une comparaison entre les articles faisant mention et ceux pour qui cela n'est pas le cas, mais bien d'une comparaison entre les articles faisant mention du mouvement. Nous n'avons pas réellement besoin des autres articles. <br><br><br><div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">Le module newspaper est utile pour les raisons suivantes : <br><br>
- Il nous évite de passer par la technique du départ, à savoir envoyer une requête avec urllib, traiter le contenu avec bs4, puis convertir en chaîne de caractères pour finalement scinder le texte à partir de balises dans le code. Newspaper fait ses étapes en une seule commande.<br>
- Le module crée un objet de type Article, ce qui lui infère de nombreux attributs. C'est pour cela que l'on ne peut pas l'utiliser lors de la première étape. <br>
</span></div>
  <br><br><br>***5.1 Interroger un site (soit le premier lien du 11 janvier 2013)***</span></div>

In [25]:
article_url = php_list[0]
print (article_url)

http://www.lapresse.ca/le-droit/actualites/ephemerides/201301/04/01-4608380-cest-arrive-un-11-janvier.php


<div style="text-align: justify"> <span style="color: black; font-family: Babas Neue; font-size: 1em;">***<br>5.1.1  Utiliser le module newspaper pour une site***</span></div>

In [29]:
article = newspaper.Article(article_url, language ='fr')
article.download()
article.parse()
print (type(article))

<class 'newspaper.article.Article'>


In [ ]:
article_data_list = []

for x in range(len(php_list)):
    print (php_list[x], '{} of {}'.format(x+1, len(php_list)))
    
    article_url = newspaper.Article(php_list[x], language ='fr')
    article_url.download()
    str_html = article.html
    article_url.parse()

    keyws = article_url.meta_keywords
    authors = article_url.authors
    if len(authors) == 0 :
        authors = None
    text = article_url.text
    title = article_url.title
    metades = article_url.meta_description

    data_array = {'date' : array_of_time[41], 'authors' : authors, 'title' : title, 'text' : text, 'meta_keywords' : keyws,
                 'meta_description' : metades, 'url' : php_list[x]}
    
    article_data_list.append(pd.Series(index=['LP_{}{}{}-{}'.format(array_of_time[41].year,
                                                                   array_of_time[41].month,
                                                                    array_of_time[41].day,
                                                                    x
                                                                   )],data=data_array))

In [ ]:
pd.concat(article_data_list, axis=1)

In [ ]:
pd.Series(data_array)